In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc numpy pandas

# Algoritmo de Shor

*Estimación de uso: Tres segundos en un procesador Eagle r3 (NOTA: Esto es solo una estimación. Su tiempo de ejecución podría variar.)*

[El algoritmo de Shor,](https://epubs.siam.org/doi/abs/10.1137/S0036144598347011) desarrollado por Peter Shor en 1994, es un algoritmo cuántico revolucionario para factorizar enteros en tiempo polinomial. Su importancia radica en su capacidad para factorizar enteros grandes exponencialmente más rápido que cualquier algoritmo clásico conocido, amenazando la seguridad de los sistemas criptográficos ampliamente utilizados como RSA, que dependen de la dificultad de factorizar números grandes. Al resolver eficientemente este problema en una computadora cuántica suficientemente potente, el algoritmo de Shor podría revolucionar campos como la criptografía, la ciberseguridad y las matemáticas computacionales, subrayando el poder transformador de la computación cuántica.

Este tutorial se centra en demostrar el algoritmo de Shor factorizando 15 en una computadora cuántica.

Primero, definimos el problema de búsqueda de orden y construimos los circuitos correspondientes a partir del protocolo de estimación de fase cuántica. A continuación, ejecutamos los circuitos de búsqueda de orden en hardware real utilizando los circuitos de menor profundidad que podemos transpilar. La última sección completa el algoritmo de Shor conectando el problema de búsqueda de orden con la factorización de enteros.

Finalizamos el tutorial con una discusión sobre otras demostraciones del algoritmo de Shor en hardware real, centrándonos tanto en implementaciones genéricas como en aquellas diseñadas específicamente para factorizar enteros específicos como 15 y 21.
Nota: Este tutorial se centra más en la implementación y demostración de los circuitos relacionados con el algoritmo de Shor. Para un recurso educativo en profundidad sobre el material, consulte el curso [Fundamentals of quantum algorithms](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction) del Dr. John Watrous, y los artículos en la sección de [Referencias](#references).
### Requisitos
Antes de comenzar este tutorial, asegúrese de tener instalado lo siguiente:
- Qiskit SDK v2.0 o posterior, con soporte de [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.40 o posterior (`pip install qiskit-ibm-runtime`)
### Configuración

In [ ]:
import numpy as np
import pandas as pd
from fractions import Fraction
from math import floor, gcd, log

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT, UnitaryGate
from qiskit.transpiler import CouplingMap, generate_preset_pass_manager
from qiskit.visualization import plot_histogram

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Paso 1: Mapear entradas clásicas a un problema cuántico
### Antecedentes
El algoritmo de Shor para la factorización de enteros utiliza un problema intermediario conocido como el problema de *búsqueda de orden*. En esta sección, demostramos cómo resolver el problema de búsqueda de orden utilizando la *estimación de fase cuántica*.
### Problema de estimación de fase
En el problema de estimación de fase, se nos proporciona un estado cuántico $\ket{\psi}$ de $n$ qubits, junto con un circuito cuántico unitario que actúa sobre $n$ qubits. Se nos garantiza que $\ket{\psi}$ es un vector propio de la matriz unitaria $U$ que describe la acción del circuito, y nuestro objetivo es calcular o aproximar el valor propio $\lambda = e^{2 \pi i \theta}$ al cual corresponde $\ket{\psi}$. En otras palabras, el circuito debería producir una aproximación del número $\theta \in [0, 1)$ que satisface $$U \ket{\psi}= e^{2 \pi i \theta} \ket{\psi}.$$
El objetivo del circuito de estimación de fase es aproximar $\theta$ en $m$ bits. Matemáticamente hablando, deseamos encontrar $y$ tal que $\theta \approx y / 2^m$, donde $y \in {0, 1, 2, \dots, 2^{m-1}}$. La siguiente imagen muestra el circuito cuántico que estima $y$ en $m$ bits realizando una medición sobre $m$ qubits.
![Quantum phase estimation circuit](../learning/images/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/phase-estimation-procedure.svg)
En el circuito anterior, los $m$ qubits superiores se inicializan en el estado $\ket{0^m}$, y los $n$ qubits inferiores se inicializan en $\ket{\psi}$, el cual se garantiza que es un vector propio de $U$. El primer ingrediente en el circuito de estimación de fase son las operaciones unitarias controladas que se encargan de realizar un *retroceso de fase* (phase kickback) en su qubit de control correspondiente. Estas unitarias controladas se exponencian de acuerdo con la posición del qubit de control, desde el bit menos significativo hasta el bit más significativo. Dado que $\ket{\psi}$ es un vector propio de $U$, el estado de los $n$ qubits inferiores no se ve afectado por esta operación, pero la información de fase del valor propio se propaga a los $m$ qubits superiores.
Resulta que después de la operación de retroceso de fase mediante unitarias controladas, todos los estados posibles de los $m$ qubits superiores son ortonormales entre sí para cada vector propio $\ket{\psi}$ del operador unitario $U$. Por lo tanto, estos estados son perfectamente distinguibles, y podemos rotar la base que forman de vuelta a la base computacional para realizar una medición. Un análisis matemático muestra que esta matriz de rotación corresponde a la transformada cuántica de Fourier (QFT) inversa en el espacio de Hilbert de dimensión $2^m$. La intuición detrás de esto es que la estructura periódica de los operadores de exponenciación modular está codificada en el estado cuántico, y la QFT convierte esta periodicidad en picos medibles en el dominio de frecuencia.

Para una comprensión más profunda de por qué se emplea el circuito QFT en el algoritmo de Shor, remitimos al lector al curso [Fundamentals of quantum algorithms](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction).
Ahora estamos listos para utilizar el circuito de estimación de fase para la búsqueda de orden.
### Problema de búsqueda de orden
Para definir el problema de búsqueda de orden, comenzamos con algunos conceptos de teoría de números. Primero, para cualquier entero positivo dado $N$, definimos el conjunto $\mathbb{Z}_N$ como $$\mathbb{Z}_N = {0, 1, 2, \dots, N-1}.$$
Todas las operaciones aritméticas en $\mathbb{Z}_N$ se realizan módulo $N$. En particular, todos los elementos $a \in \mathbb{Z}_n$ que son coprimos con $N$ son especiales y constituyen $\mathbb{Z}^*_N$ como $$\mathbb{Z}^*_N = { a \in \mathbb{Z}_N : \mathrm{gcd}(a, N)=1 }.$$
Para un elemento $a \in \mathbb{Z}^*_N$, el entero positivo más pequeño $r$ tal que $$a^r \equiv 1 \; (\mathrm{mod} \; N)$$ se define como el *orden* de $a$ módulo $N$. Como veremos más adelante, encontrar el orden de un $a \in \mathbb{Z}^*_N$ nos permitirá factorizar $N$.
Para construir el circuito de búsqueda de orden a partir del circuito de estimación de fase, necesitamos dos consideraciones. Primero, necesitamos definir el operador unitario $U$ que nos permitirá encontrar el orden $r$, y segundo, necesitamos definir un vector propio $\ket{\psi}$ de $U$ para preparar el estado inicial del circuito de estimación de fase.

Para conectar el problema de búsqueda de orden con la estimación de fase, consideramos la operación definida sobre un sistema cuyos estados clásicos corresponden a $\mathbb{Z}_N$, donde multiplicamos por un elemento fijo $a \in \mathbb{Z}^*_N$. En particular, definimos este operador de multiplicación $M_a$ tal que $$M_a \ket{x} = \ket{ax \; (\mathrm{mod} \; N)}$$ para cada $x \in \mathbb{Z}_N$. Observe que está implícito que estamos tomando el producto módulo $N$ dentro del ket en el lado derecho de la ecuación. Un análisis matemático muestra que $M_a$ es un operador unitario. Además, resulta que $M_a$ tiene pares de vectores propios y valores propios que nos permiten conectar el orden $r$ de $a$ con el problema de estimación de fase. Específicamente, para cualquier elección de $j \in {0, \dots, r-1}$, tenemos que $$\ket{\psi_j} = \frac{1}{\sqrt{r}} \sum^{r-1}_{k=0} \omega^{-jk}_{r} \ket{a^k}$$ es un vector propio de $M_a$ cuyo valor propio correspondiente es $\omega^{j}_{r}$, donde $$\omega^{j}_{r} = e^{2 \pi i \frac{j}{r}}.$$
Por observación, vemos que un par conveniente de vector propio/valor propio es el estado $\ket{\psi_1}$ con $\omega^{1}_{r} = e^{2 \pi i \frac{1}{r}}$. Por lo tanto, si pudiéramos encontrar el vector propio $\ket{\psi_1}$, podríamos estimar la fase $\theta=1/r$ con nuestro circuito cuántico y así obtener una estimación del orden $r$. Sin embargo, no es fácil hacerlo, y necesitamos considerar una alternativa.

Consideremos qué resultado produciría el circuito si preparamos el estado computacional $\ket{1}$ como estado inicial. Este no es un estado propio de $M_a$, pero es la superposición uniforme de los estados propios que acabamos de describir. En otras palabras, se cumple la siguiente relación: $$ \ket{1} = \frac{1}{\sqrt{r}} \sum^{r-1}_{k=0} \ket{\psi_k} $$
La implicación de la ecuación anterior es que si establecemos el estado inicial como $\ket{1}$, obtendremos precisamente el mismo resultado de medición que si hubiéramos elegido $k \in { 0, \dots, r-1}$ uniformemente al azar y hubiéramos usado $\ket{\psi_k}$ como vector propio en el circuito de estimación de fase. En otras palabras, una medición de los $m$ qubits superiores produce una aproximación $y / 2^m$ al valor $k / r$ donde $k \in { 0, \dots, r-1}$ se elige uniformemente al azar. Esto nos permite aprender $r$ con un alto grado de confianza después de varias ejecuciones independientes, que era nuestro objetivo.
### Operadores de exponenciación modular
Hasta ahora, vinculamos el problema de estimación de fase con el problema de búsqueda de orden definiendo $U = M_a$ y $\ket{\psi} = \ket{1}$ en nuestro circuito cuántico. Por lo tanto, el último ingrediente restante es encontrar una forma eficiente de definir las exponenciales modulares de $M_a$ como $M_a^k$ para $k = 1, 2, 4, \dots, 2^{m-1}$.
Para realizar este cálculo, encontramos que para cualquier potencia $k$ que elijamos, podemos crear un circuito para $M_a^k$ no iterando $k$ veces el circuito para $M_a$, sino calculando $b = a^k \; \mathrm{mod} \; N$ y luego usando el circuito para $M_b$. Dado que solo necesitamos las potencias que son a su vez potencias de 2, podemos hacer esto clásicamente de forma eficiente usando la elevación al cuadrado iterativa.
## Paso 2: Optimizar el problema para la ejecución en hardware cuántico
### Ejemplo específico con $N = 15$ y $a=2$
Podemos hacer una pausa aquí para discutir un ejemplo específico y construir el circuito de búsqueda de orden para $N=15$. Observe que los posibles valores no triviales de $a \in \mathbb{Z}_N^*$ para $N=15$ son $a \in {2, 4, 7, 8, 11, 13, 14 }$. Para este ejemplo, elegimos $a=2$. Construiremos el operador $M_2$ y los operadores de exponenciación modular $M_2^k$.
La acción de $M_2$ sobre los estados de la base computacional es la siguiente.
$$M_2 \ket{0} = \ket{0} \quad M_2 \ket{5} = \ket{10} \quad M_2 \ket{10} = \ket{5}$$
$$M_2 \ket{1} = \ket{2} \quad M_2 \ket{6} = \ket{12} \quad M_2 \ket{11} = \ket{7}$$
$$M_2 \ket{2} = \ket{4} \quad M_2 \ket{7} = \ket{14} \quad M_2 \ket{12} = \ket{9}$$
$$M_2 \ket{3} = \ket{6} \quad M_2 \ket{8} = \ket{1} \quad M_2 \ket{13} = \ket{11}$$
$$M_2 \ket{4} = \ket{8} \quad M_2 \ket{9} = \ket{3} \quad M_2 \ket{14} = \ket{13}$$
Por observación, podemos ver que los estados de la base se reorganizan, por lo que tenemos una matriz de permutación. Podemos construir esta operación en cuatro qubits con compuertas de intercambio (swap). A continuación, construimos las operaciones $M_2$ y $M_2$ controlada.

In [2]:
def M2mod15():
    """
    M2 (mod 15)
    """
    b = 2
    U = QuantumCircuit(4)

    U.swap(2, 3)
    U.swap(1, 2)
    U.swap(0, 1)

    U = U.to_gate()
    U.name = f"M_{b}"

    return U

In [3]:
# Get the M2 operator
M2 = M2mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(4)
circ.compose(M2, inplace=True)
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0a8885f1-91d4-40bd-912d-dc5eea05f5bd-0.avif" alt="Output of the previous code cell" />

In [4]:
def controlled_M2mod15():
    """
    Controlled M2 (mod 15)
    """
    b = 2
    U = QuantumCircuit(4)

    U.swap(2, 3)
    U.swap(1, 2)
    U.swap(0, 1)

    U = U.to_gate()
    U.name = f"M_{b}"
    c_U = U.control()

    return c_U

In [5]:
# Get the controlled-M2 operator
controlled_M2 = controlled_M2mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(5)
circ.compose(controlled_M2, inplace=True)
circ.decompose(reps=1).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/ab7fe331-2f9e-47ca-ba3b-f5d67992062a-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/ab7fe331-2f9e-47ca-ba3b-f5d67992062a-0.avif)

Las compuertas que actúan sobre más de dos qubits se descompondrán adicionalmente en compuertas de dos qubits.

In [6]:
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/13b4841d-a4ac-46bd-b4d0-d111b3017189-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/13b4841d-a4ac-46bd-b4d0-d111b3017189-0.avif)

Ahora necesitamos construir los operadores de exponenciación modular. Para obtener suficiente precisión en la estimación de fase, utilizaremos ocho qubits para la medición de estimación. Por lo tanto, necesitamos construir $M_b$ con $b = a^{2^k} \; (\mathrm{mod} \; N)$ para cada $k = 0, 1, \dots, 7$.

In [7]:
def a2kmodN(a, k, N):
    """Compute a^{2^k} (mod N) by repeated squaring"""
    for _ in range(k):
        a = int(np.mod(a**2, N))
    return a

In [8]:
k_list = range(8)
b_list = [a2kmodN(2, k, 15) for k in k_list]

print(b_list)

[2, 4, 1, 1, 1, 1, 1, 1]


As we can see from the list of $b$ values, in addition to $M_2$ that we previously constructed, we also need to build $M_4$ and $M_1$. Note that $M_1$ acts trivially on the computational basis states, so it is simply the identity operator.

$M_4$ acts on the computational basis states as follows.
$$M_4 \ket{0} = \ket{0} \quad M_4 \ket{5} = \ket{5} \quad M_4 \ket{10} = \ket{10}$$
$$M_4 \ket{1} = \ket{4} \quad M_4 \ket{6} = \ket{9} \quad M_4 \ket{11} = \ket{14}$$
$$M_4 \ket{2} = \ket{8} \quad M_4 \ket{7} = \ket{13} \quad M_4 \ket{12} = \ket{3}$$
$$M_4 \ket{3} = \ket{12} \quad M_4 \ket{8} = \ket{2} \quad M_4 \ket{13} = \ket{7}$$
$$M_4 \ket{4} = \ket{1} \quad M_4 \ket{9} = \ket{6} \quad M_4 \ket{14} = \ket{11}$$

Therefore, this permutation can be constructed with the following swap operation.

In [9]:
def M4mod15():
    """
    M4 (mod 15)
    """
    b = 4
    U = QuantumCircuit(4)

    U.swap(1, 3)
    U.swap(0, 2)

    U = U.to_gate()
    U.name = f"M_{b}"

    return U

In [10]:
# Get the M4 operator
M4 = M4mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(4)
circ.compose(M4, inplace=True)
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/be041e3d-28b1-453e-983e-184c2366aeb9-0.avif" alt="Output of the previous code cell" />

In [11]:
def controlled_M4mod15():
    """
    Controlled M4 (mod 15)
    """
    b = 4
    U = QuantumCircuit(4)

    U.swap(1, 3)
    U.swap(0, 2)

    U = U.to_gate()
    U.name = f"M_{b}"
    c_U = U.control()

    return c_U

In [12]:
# Get the controlled-M4 operator
controlled_M4 = controlled_M4mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(5)
circ.compose(controlled_M4, inplace=True)
circ.decompose(reps=1).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/8d943b00-a502-4157-8a0d-13fb1f55e705-0.avif" alt="Output of the previous code cell" />

Gates acting on more than two qubits will be further decomposed into two-qubit gates.

In [13]:
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/68399eef-5e55-4c95-a8a4-c8efaebd34b9-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/8d943b00-a502-4157-8a0d-13fb1f55e705-0.avif)

Las compuertas que actúan sobre más de dos qubits se descompondrán adicionalmente en compuertas de dos qubits.

In [14]:
def mod_mult_gate(b, N):
    """
    Modular multiplication gate from permutation matrix.
    """
    if gcd(b, N) > 1:
        print(f"Error: gcd({b},{N}) > 1")
    else:
        n = floor(log(N - 1, 2)) + 1
        U = np.full((2**n, 2**n), 0)
        for x in range(N):
            U[b * x % N][x] = 1
        for x in range(N, 2**n):
            U[x][x] = 1
        G = UnitaryGate(U)
        G.name = f"M_{b}"
        return G

In [15]:
# Let's build M2 using the permutation matrix definition
M2_other = mod_mult_gate(2, 15)

# Add it to a circuit
circ = QuantumCircuit(4)
circ.compose(M2_other, inplace=True)
circ = circ.decompose()

# Transpile the circuit and get the depth
coupling_map = CouplingMap.from_line(4)
pm = generate_preset_pass_manager(coupling_map=coupling_map)
transpiled_circ = pm.run(circ)

print(f"qubits: {circ.num_qubits}")
print(
    f"2q-depth: {transpiled_circ.depth(lambda x: x.operation.num_qubits==2)}"
)
print(f"2q-size: {transpiled_circ.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_circ.count_ops()}")
transpiled_circ.decompose().draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

qubits: 4
2q-depth: 94
2q-size: 96
Operator counts: OrderedDict({'cx': 45, 'swap': 32, 'u': 24, 'u1': 7, 'u3': 4, 'unitary': 3, 'circuit-335': 1, 'circuit-338': 1, 'circuit-341': 1, 'circuit-344': 1, 'circuit-347': 1, 'circuit-350': 1, 'circuit-353': 1, 'circuit-356': 1, 'circuit-359': 1, 'circuit-362': 1, 'circuit-365': 1, 'circuit-368': 1, 'circuit-371': 1, 'circuit-374': 1, 'circuit-377': 1, 'circuit-380': 1})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/c184f6dd-9f80-4487-ac0b-0dd94170b0f0-1.avif" alt="Output of the previous code cell" />

Let's compare these counts with the compiled circuit depth of our manual implementation of the $M_2$ gate.

In [16]:
# Get the M2 operator from our manual construction
M2 = M2mod15()

# Add it to a circuit
circ = QuantumCircuit(4)
circ.compose(M2, inplace=True)
circ = circ.decompose(reps=3)

# Transpile the circuit and get the depth
coupling_map = CouplingMap.from_line(4)
pm = generate_preset_pass_manager(coupling_map=coupling_map)
transpiled_circ = pm.run(circ)

print(f"qubits: {circ.num_qubits}")
print(
    f"2q-depth: {transpiled_circ.depth(lambda x: x.operation.num_qubits==2)}"
)
print(f"2q-size: {transpiled_circ.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_circ.count_ops()}")
transpiled_circ.draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

qubits: 4
2q-depth: 9
2q-size: 9
Operator counts: OrderedDict({'cx': 9})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0235c931-0adb-4972-9fce-32a0341822bf-1.avif" alt="Output of the previous code cell" />

As we can see, the permutation matrix approach resulted in a significantly deep circuit even for a single $M_2$ gate compared to our manual implementation of it. Therefore, we will continue with our previous implementation of the $M_b$ operations.

Now, we are ready to construct the full order finding circuit using our previously defined controlled modular exponentiation operators. In the following code, we also import the [QFT circuit](/docs/api/qiskit/qiskit.circuit.library.QFT) from the Qiskit Circuit library, which uses Hadamard gates on each qubit, a series of controlled-U1 (or Z, depending on the phase) gates, and a layer of swap gates.

In [17]:
# Order finding problem for N = 15 with a = 2
N = 15
a = 2

# Number of qubits
num_target = floor(log(N - 1, 2)) + 1  # for modular exponentiation operators
num_control = 2 * num_target  # for enough precision of estimation

# List of M_b operators in order
k_list = range(num_control)
b_list = [a2kmodN(2, k, 15) for k in k_list]

# Initialize the circuit
control = QuantumRegister(num_control, name="C")
target = QuantumRegister(num_target, name="T")
output = ClassicalRegister(num_control, name="out")
circuit = QuantumCircuit(control, target, output)

# Initialize the target register to the state |1>
circuit.x(num_control)

# Add the Hadamard gates and controlled versions of the
# multiplication gates
for k, qubit in enumerate(control):
    circuit.h(k)
    b = b_list[k]
    if b == 2:
        circuit.compose(
            M2mod15().control(), qubits=[qubit] + list(target), inplace=True
        )
    elif b == 4:
        circuit.compose(
            M4mod15().control(), qubits=[qubit] + list(target), inplace=True
        )
    else:
        continue  # M1 is the identity operator

# Apply the inverse QFT to the control register
circuit.compose(QFT(num_control, inverse=True), qubits=control, inplace=True)

# Measure the control register
circuit.measure(control, output)

circuit.draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0e854aed-c11b-494c-8c80-adeb8eb0e8fe-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/c184f6dd-9f80-4487-ac0b-0dd94170b0f0-1.avif)

Comparemos estos conteos con la profundidad del circuito compilado de nuestra implementación manual de la compuerta $M_2$.

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_marrakesh")
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)

transpiled_circuit = pm.run(circuit)

print(
    f"2q-depth: {transpiled_circuit.depth(lambda x: x.operation.num_qubits==2)}"
)
print(
    f"2q-size: {transpiled_circuit.size(lambda x: x.operation.num_qubits==2)}"
)
print(f"Operator counts: {transpiled_circuit.count_ops()}")
transpiled_circuit.draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

2q-depth: 187
2q-size: 260
Operator counts: OrderedDict({'sx': 521, 'rz': 354, 'cz': 260, 'measure': 8, 'x': 4})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/95925dd5-7ba9-4746-b96e-ba50400fa5ac-1.avif" alt="Output of the previous code cell" />

## Step 3: Execute using Qiskit primitives

First, we discuss what we would theoretically obtain if we ran this circuit on an ideal simulator. Below, we have a set of simulation results of the above circuit using 1024 shots. As we can see, we get an approximately uniform distribution over four bitstrings over the control qubits.

In [19]:
# Obtained from the simulator
counts = {"00000000": 264, "01000000": 268, "10000000": 249, "11000000": 243}

In [20]:
plot_histogram(counts)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0d6d2702-02e4-47de-8f7e-0b256657ef0f-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/0e854aed-c11b-494c-8c80-adeb8eb0e8fe-0.avif)

Observe que omitimos las operaciones de exponenciación modular controlada de los qubits de control restantes porque $M_1$ es el operador identidad.
Tenga en cuenta que más adelante en este tutorial, ejecutaremos este circuito en el backend `ibm_marrakesh`. Para ello, transpilamos el circuito de acuerdo con este backend específico e informamos la profundidad del circuito y los conteos de compuertas.

In [21]:
# Rows to be displayed in table
rows = []
# Corresponding phase of each bitstring
measured_phases = []

for output in counts:
    decimal = int(output, 2)  # Convert bitstring to decimal
    phase = decimal / (2**num_control)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append(
        [
            f"{output}(bin) = {decimal:>3}(dec)",
            f"{decimal}/{2 ** num_control} = {phase:.2f}",
        ]
    )

# Print the rows in a table
headers = ["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

            Register Output           Phase
0  00000000(bin) =   0(dec)    0/256 = 0.00
1  01000000(bin) =  64(dec)   64/256 = 0.25
2  10000000(bin) = 128(dec)  128/256 = 0.50
3  11000000(bin) = 192(dec)  192/256 = 0.75


Recall that the any measured phase corresponds to $\theta = k / r$ where $k$ is sampled uniformly at random from $\{0, 1, \dots, r-1 \}$. Therefore, we can use the continued fractions algorithm to attempt to find $k$ and the order $r$. Python has this functionality built in. We can use the `fractions` module to turn a float into a `Fraction` object, for example:

In [22]:
Fraction(0.666)

Fraction(5998794703657501, 9007199254740992)

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/95925dd5-7ba9-4746-b96e-ba50400fa5ac-1.avif)
## Paso 3: Ejecutar utilizando primitivas de Qiskit
Primero, discutimos lo que teóricamente obtendríamos si ejecutáramos este circuito en un simulador ideal. A continuación, tenemos un conjunto de resultados de simulación del circuito anterior utilizando 1024 disparos. Como podemos ver, obtenemos una distribución aproximadamente uniforme sobre cuatro cadenas de bits en los qubits de control.

In [23]:
# Get fraction that most closely resembles 0.666
# with denominator < 15
Fraction(0.666).limit_denominator(15)

Fraction(2, 3)

This is much nicer. The order (r) must be less than N, so we will set the maximum denominator to be `15`:

In [24]:
# Rows to be displayed in a table
rows = []

for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append(
        [phase, f"{frac.numerator}/{frac.denominator}", frac.denominator]
    )

# Print the rows in a table
headers = ["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)

   Phase Fraction  Guess for r
0   0.00      0/1            1
1   0.25      1/4            4
2   0.50      1/2            2
3   0.75      3/4            4


![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/0d6d2702-02e4-47de-8f7e-0b256657ef0f-0.avif)

Al medir los qubits de control, obtenemos una estimación de fase de ocho bits del operador $M_a$. Podemos convertir esta representación binaria a decimal para encontrar la fase medida. Como podemos ver en el histograma anterior, se midieron cuatro cadenas de bits diferentes, y cada una de ellas corresponde a un valor de fase de la siguiente manera.

In [ ]:
# Sampler primitive to obtain the probability distribution
sampler = Sampler(backend)

# Turn on dynamical decoupling with sequence XpXm
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"
# Enable gate twirling
sampler.options.twirling.enable_gates = True

pub = transpiled_circuit
job = sampler.run([pub], shots=1024)

In [25]:
result = job.result()[0]
counts = result.data["out"].get_counts()

In [26]:
plot_histogram(counts, figsize=(35, 5))

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/559d7030-1f67-44e8-afa7-6afc7a334677-0.avif" alt="Output of the previous code cell" />

As we can see, we obtained the same bitstrings with highest counts. Since quantum hardware has noise, there is some leakage to other bitstrings, which we can filter out statistically.

In [27]:
# Dictionary of bitstrings and their counts to keep
counts_keep = {}
# Threshold to filter
threshold = np.max(list(counts.values())) / 2

for key, value in counts.items():
    if value > threshold:
        counts_keep[key] = value

print(counts_keep)

{'00000000': 58, '01000000': 41, '11000000': 42, '10000000': 40}


Debido a que esto produce fracciones que devuelven el resultado exacto (en este caso, `0.6660000...`), puede dar resultados complicados como el anterior. Podemos usar el método `.limit_denominator()` para obtener la fracción que más se asemeja a nuestro número de punto flotante, con un denominador por debajo de cierto valor:

In [28]:
a = 2
N = 15

FACTOR_FOUND = False
num_attempt = 0

while not FACTOR_FOUND:
    print(f"\nATTEMPT {num_attempt}:")
    # Here, we get the bitstring by iterating over outcomes
    # of a previous hardware run with multiple shots.
    # Instead, we can also perform a single-shot measurement
    # here in the loop.
    bitstring = list(counts_keep.keys())[num_attempt]
    num_attempt += 1
    # Find the phase from measurement
    decimal = int(bitstring, 2)
    phase = decimal / (2**num_control)  # phase = k / r
    print(f"Phase: theta = {phase}")

    # Guess the order from phase
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator  # order = r
    print(f"Order of {a} modulo {N} estimated as: r = {r}")

    if phase != 0:
        # Guesses for factors are gcd(a^{r / 2} ± 1, 15)
        if r % 2 == 0:
            x = pow(a, r // 2, N) - 1
            d = gcd(x, N)
            if d > 1:
                FACTOR_FOUND = True
                print(f"*** Non-trivial factor found: {x} ***")


ATTEMPT 0:
Phase: theta = 0.0
Order of 2 modulo 15 estimated as: r = 1

ATTEMPT 1:
Phase: theta = 0.25
Order of 2 modulo 15 estimated as: r = 4
*** Non-trivial factor found: 3 ***


## Discussion

### Related work
In this section, we discuss other milestone work that has demonstrated Shor's algorithm on real hardware.

The seminal work [[3]](#references) from IBM&reg; demonstrated Shor's algorithm for the first time, factoring the number 15 into its prime factors 3 and 5 using a seven-qubit nuclear magnetic resonance (NMR) quantum computer. Another experiment [[4]](#references) factored 15 using photonic qubits. By employing a single qubit recycled multiple times and encoding the work register in higher-dimensional states, the researchers reduced the required number of qubits to one-third of that in the standard protocol, utilizing a two-photon compiled algorithm. A significant paper in the demonstration of Shor's algorithm is [[5]](#references), which uses Kitaev's iterative phase estimation [[8]](#references) technique to reduce the qubit requirement of the algorithm. Authors used seven control qubits and four cache qubits, together with the implementation of modular multipliers. This implementation, however, requires mid-circuit measurements with feed-forward operations and qubit recycling with reset operations. This demonstration was done on an ion-trap quantum computer.

More recent work [[6]](#references) focused on factoring 15, 21, and 35 on IBM Quantum&reg; hardware. Similar to previous work, researchers used a compiled version of the algorithm that employed a semi-classical quantum Fourier transform as proposed by Kitaev to minimize the number of physical qubits and gates. A most recent work [[7]](#references) also performed a proof-of-concept demonstration for factoring the integer 21. This demonstration also involved the use of a compiled version of the quantum phase estimation routine, and built upon the previous demonstration by [[4]](#references). Authors went beyond this work by using a configuration of approximate Toffoli gates with residual phase shifts. The algorithm was implemented on IBM quantum processors using only five qubits, and the presence of entanglement between the control and register qubits was verified successfully.

### Scaling of the algorithm

We note that RSA encryption typically involves key sizes on the order of 2048 to 4096 bits. Attempting to factor a 2048-bit number with Shor's algorithm will result in a quantum circuit with millions of qubits, including the error correction overhead and a circuit depth on the order of a billion, which is beyond the limits of current quantum hardware to execute. Therefore, Shor's algorithm will require either optimized circuit construction methods or robust quantum error correction to be practically viable for breaking modern cryptographic systems. We refer you to [[9]](#references) for a more detailed discussion on resource estimation for Shor's algorithm.

## Challenge

Congratulations for finishing the tutorial! Now is a great time to test your understanding. Could you try to construct the circuit for factoring 21? You can select an $a$ of your own choice. You will need to decide on the bit accuracy of the algorithm to choose the number of qubits, and you will need to design the modular exponentiation operators $M_a$. We encourage you to try this out yourself, and then read about the methodologies shown in Fig. 9 of [[6]](#references) and Fig. 2 of [[7]](#references).

In [ ]:
def M_a_mod21():
    """
    M_a (mod 21)
    """

    # Your code here
    pass

Esto es mucho mejor. El orden (r) debe ser menor que N, por lo que estableceremos el denominador máximo en `15`: